# Sentence Classification with BERT

In this lab we turn a pre-trained BERT model into a trainable Keras layer and apply it to the semeval 2017 task 4 subtask B that we tackled in lab 5. BERT (Bidirectional Embedding Representations from Transformers) is a new model for pre-training language representations that obtains state-of-the-art results on many NLP tasks. We demonstrate how to integrate BERT as a custom Keras layer to simplify model prototyping using huggingface. In this lab, you will learn: 

1) How to use the huggingface package.

2) How to integrate BERT in our previous model. 

3) How to use the TPU from Colab. (Note: Running the BERT on the CPU would be very slow. Thus we recommend you to do this lab on Colab based on TPU provided by Google.)

In [1]:
import keras
import numpy as np
from keras.layers import Lambda, GlobalAveragePooling1D, Dense, Embedding
from keras import backend as K
from keras.models import Sequential
import matplotlib.pyplot as plt

from keras.layers import LSTM, RNN, Dropout, Input, LeakyReLU, Bidirectional,Conv1D, GlobalMaxPooling1D
from keras.layers.core import Dense
from keras.models import Model
from keras.utils import plot_model


Before we start, we should install the huggingface transformer package. You can find the doc from its [website](https://huggingface.co/transformers/index.html).

In [2]:
pip install transformers

     |████████████████████████████████| 2.1MB 6.9MB/s 
     |████████████████████████████████| 3.3MB 44.7MB/s 
     |████████████████████████████████| 870kB 49.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=03bc6cba106367eb7586240367b7df1337173993cbf58cf44b660054f4772769
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


## Preprocessing and Tokenization

In this lab we will use DistilBERT instead of BERT: DistilBERT is a small, fast, cheap and light Transformer model trained by distilling BERT base. It has 40% less parameters than bert-base-uncased, and runs 60% faster, while preserving over 95% of BERT’s performance as measured on the GLUE language understanding benchmark.

It is easy to switch between DistilBERT and BERT using the huggingface transformer package. This huggingface package provides many pre-trained and pre-built models that are easy to use via a few lines of code.

Before using DistilBERT or BERT, we need a tokenizer. Generally speaking, every BERT related model has its own tokenizer, trained for that model (see this week's lecture video on sub-word tokenization). 
We can get the DistilBERT tokenizer from **DistilBertTokenizer.from_pretrained** function.

In [3]:
from transformers import DistilBertTokenizer, RobertaTokenizer 
import tqdm
distil_bert = 'distilbert-base-uncased' # Pick any desired pre-trained model

# Defining DistilBERT tokonizer
tokenizer = DistilBertTokenizer.from_pretrained(distil_bert, do_lower_case=True, add_special_tokens=True,
                                                max_length=128, pad_to_max_length=True)

def tokenize(sentences, tokenizer, pad_length=128, pad_to_max_length=True ):
    if type(sentences) == str:
        inputs = tokenizer.encode_plus(sentences, add_special_tokens=True, max_length=pad_length, pad_to_max_length=pad_to_max_length, 
                                             return_attention_mask=True, return_token_type_ids=True)
        return np.asarray(inputs['input_ids'], dtype='int32'), np.asarray(inputs['attention_mask'], dtype='int32'), np.asarray(inputs['token_type_ids'], dtype='int32')
    input_ids, input_masks, input_segments = [],[],[]
    for sentence in sentences:
        inputs = tokenizer.encode_plus(sentence, add_special_tokens=True, max_length=pad_length, pad_to_max_length=pad_to_max_length, 
                                             return_attention_mask=True, return_token_type_ids=True)
        input_ids.append(inputs['input_ids'])
        input_masks.append(inputs['attention_mask'])
        input_segments.append(inputs['token_type_ids'])        
        
    return np.asarray(input_ids, dtype='int32'), np.asarray(input_masks, dtype='int32'), np.asarray(input_segments, dtype='int32')


Then we can use the tokenizer to tokenize the sentence. When working with word2vec and GloVe, we tokenized the sentence into words ourselves and then converted the tokens to GloVe word indices. But in BERT, we must use the BERT tokenizer: the tokens for BERT are different, and include whole words and sub-word tokens (see lecture video on sub-word tokenisation).

For example, for the sentence: **This is a pretrained model.** our previous word-based tokenizer will generate the following tokens:

**"this", "is", "a", "pretrained", "model", "."**

Then you will find out that the word token "pretrained" is not in the GloVe word dictionary. Thus we can not assign a proper word vector for "pretrained".

In BERT, the BERT tokenizer will separate the word "pretrained" into three sub-word tokens:

**'pre', '##train', '##ed'**

This way, BERT can use these three token vectors to represent the word "pretrained". Without the BERT tokenizer, it is hard to separate these unknown words properly.

In [4]:
inputs = tokenizer.tokenize("The capital of France is [MASK].")
print(inputs,'\n')

inputs = tokenizer.tokenize("This is a pretrained model.")
print(inputs,'\n')

ids,masks,segments = tokenize("The capital of France is [MASK].", tokenizer)
print(ids)
print(masks,"\n")

ids,masks,segments = tokenize("The capital of France is [SEP].", tokenizer, pad_to_max_length=False)
print(ids)
print(masks)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


['the', 'capital', 'of', 'france', 'is', '[MASK]', '.'] 

['this', 'is', 'a', 'pre', '##train', '##ed', 'model', '.'] 

[ 101 1996 3007 1997 2605 2003  103 1012  102    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
[1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


## Loading Data

### Downloading and preprocessing data

Similar to lab 5, we need to download and preprocess the data first. The data download code is consistent with lab 5.

In [5]:
import requests
def downloadfile(url):
  rq = requests.get(url)
  open(url.split('/')[-1], 'wb').write(rq.content)

downloadfile('https://raw.githubusercontent.com/cbaziotis/datastories-semeval2017-task4/master/dataset/Subtask_BD/downloaded/twitter-2016devtest-BD.tsv')
downloadfile('https://raw.githubusercontent.com/cbaziotis/datastories-semeval2017-task4/master/dataset/Subtask_BD/downloaded/twitter-2016dev-BD.tsv')
downloadfile('https://raw.githubusercontent.com/cbaziotis/datastories-semeval2017-task4/master/dataset/Subtask_BD/downloaded/twitter-2016test-BD.tsv')
downloadfile('https://raw.githubusercontent.com/cbaziotis/datastories-semeval2017-task4/master/dataset/Subtask_BD/downloaded/twitter-2016train-BD.tsv')
downloadfile('https://raw.githubusercontent.com/cbaziotis/datastories-semeval2017-task4/master/dataset/Subtask_BD/downloaded/twitter-2015test-BD.tsv')
downloadfile('https://raw.githubusercontent.com/cbaziotis/datastories-semeval2017-task4/master/dataset/Subtask_BD/downloaded/twitter-2015train-BD.tsv')

downloadfile('https://raw.githubusercontent.com/cbaziotis/datastories-semeval2017-task4/master/dataset/Subtask_BD/gold/SemEval2017-task4-test.subtask-BD.english.txt')

with open('twitter-2016dev-BD.tsv', 'r') as f:
  dev_original = [l.strip().split('\t') for l in f.readlines()]
with open('SemEval2017-task4-test.subtask-BD.english.txt', 'r') as f:
  test_original = [l.strip().split('\t') for l in f.readlines()]
train_original = []
with open('twitter-2016train-BD.tsv', 'r') as f:
  train_original = [l.strip().split('\t') for l in f.readlines()]
with open('twitter-2016test-BD.tsv', 'r') as f:
  train_original.extend([l.strip().split('\t') for l in f.readlines()])
with open('twitter-2016devtest-BD.tsv', 'r') as f:
  train_original.extend([l.strip().split('\t') for l in f.readlines()])
with open('twitter-2015test-BD.tsv', 'r') as f:
  train_original.extend([l.strip().split('\t') for l in f.readlines() if l.strip().split('\t')[2] in ['negative','positive']])
with open('twitter-2015train-BD.tsv', 'r') as f:
  train_original.extend([l.strip().split('\t') for l in f.readlines() if l.strip().split('\t')[2] in ['negative','positive']])

print("Training entries: {}".format(len(train_original)))
print("Development entries: {}".format(len(dev_original)))
print("Testing entries: {}".format(len(test_original)))

Training entries: 17639
Development entries: 1325
Testing entries: 6185


We now can start playing around with the data, let’s first see some examples:

In [6]:
print("ID \t TOPIC \t LABLE \t TWEET_TEXT")
print(train_original[0])
print(train_original[1])
print(train_original[2])
print(train_original[3])
print(train_original[4])

ID 	 TOPIC 	 LABLE 	 TWEET_TEXT
['628949369883000832', '@microsoft', 'negative', "dear @Microsoft the newOoffice for Mac is great and all, but no Lync update? C'mon."]
['628976607420645377', '@microsoft', 'negative', "@Microsoft how about you make a system that doesn't eat my friggin discs. This is the 2nd time this has happened and I am so sick of it!"]
['629023169169518592', '@microsoft', 'negative', "I may be ignorant on this issue but... should we celebrate @Microsoft's parental leave changes? Doesn't the gender divide suggest... (1/2)"]
['629179223232479232', '@microsoft', 'negative', 'Thanks to @microsoft, I just may be switching over to @apple.']
['629226490152914944', '@microsoft', 'positive', 'Microsoft, I may not prefer your gaming branch of business. But, you do make a damn fine operating system. #Windows10 @Microsoft']


According to the BERT tokenize function above, we can convert the tweet text and topic words to integers:

(Note: as explained above, the BERT tokenize function is different from lab 5.)


In [7]:
# Please write your code to generate the following data
x_train_tweet_int = []
x_train_tweet_masks = []
x_train_topic_int = []
x_train_topic_masks = []
for record in train_original:
  ids,masks,segments = tokenize(record[3], tokenizer, pad_to_max_length=False)
  x_train_tweet_int.append(ids)
  x_train_tweet_masks.append(masks)
  ids,masks,segments = tokenize(record[1], tokenizer, pad_to_max_length=False)
  x_train_topic_int.append(ids)
  x_train_topic_masks.append(masks)


x_dev_tweet_int = []
x_dev_tweet_masks = []
x_dev_topic_int = []
x_dev_topic_masks = []
for record in dev_original:
  ids,masks,segments = tokenize(record[3], tokenizer, pad_to_max_length=False)
  x_dev_tweet_int.append(ids)
  x_dev_tweet_masks.append(masks)
  ids,masks,segments = tokenize(record[1], tokenizer, pad_to_max_length=False)
  x_dev_topic_int.append(ids)
  x_dev_topic_masks.append(masks)

x_test_tweet_int = []
x_test_tweet_masks = []
x_test_topic_int = []
x_test_topic_masks = []
for record in test_original:
  ids,masks,segments = tokenize(record[3], tokenizer, pad_to_max_length=False)
  x_test_tweet_int.append(ids)
  x_test_tweet_masks.append(masks)
  ids,masks,segments = tokenize(record[1], tokenizer, pad_to_max_length=False)
  x_test_topic_int.append(ids)
  x_test_topic_masks.append(masks)


# your code goes here


# If use the previous tokenize function, you can get a print result like:
assert len(x_train_topic_int) == len(train_original)
assert len(x_train_topic_masks) == len(x_train_topic_int)
assert len(x_test_topic_int) == len(test_original)
assert len(x_test_topic_masks) == len(x_test_topic_int)
print("x_dev_topic_int[0]:")
print(x_dev_topic_int[0])
print("x_dev_topic_masks[0]:")
print(x_dev_topic_masks[0])
print("x_dev_tweet_int[0]:")
print(x_dev_tweet_int[0])
print("x_dev_tweet_masks[0]:")
print(x_dev_tweet_masks[0])

x_dev_topic_int[0]:
[ 101 2745 4027  102]
x_dev_topic_masks[0]:
[1 1 1 1]
x_dev_tweet_int[0]:
[  101  6108  1062  9794 16021 23091  2007 16839  9080 12863  7050  2000
  2745  4027  1024  6108  1062  4593  2587 16021 23091  2006  5095  1998
  1012  1012  8299  1024  1013  1013  1056  1012  2522  1013  1053  3501
  2683  2072  2549  8586  2615 18037   102]
x_dev_tweet_masks[0]:
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1]


<font color = "#75a2eb"><b>In the above code we have passed each tweet and topic into the BERT Tokenize function. Along with that we also passed the Distillbert tokenizer object that we created earlier. We can pass either pad_to_max_length as False or True depending on our need. This tokenizer function returns us 3 things - The IDs, segments and masks. We then appended the ids and mask returned into our separate lists. This was done for all 3 sets - training, testing and validation.</b></font>

We use 1 to represent "positive" and 0 for "negative" and generate the "y" data similar to previous labs.

In [8]:
def label2int(dataset):
  y = []
  for example in dataset:
    if example[2].lower() == "negative":
      y.append(0)
    else:
      y.append(1)
  return y
  
y_train = label2int(train_original)
y_dev = label2int(dev_original)
y_test = label2int(test_original)
y_train = np.array(y_train)
y_dev = np.array(y_dev)
y_test = np.array(y_test)

print(y_train[0])
print(y_train[1])
print(y_train[2])
print(y_train[3])
print(y_train[4])

0
0
0
0
1


We also generate these target labels using one hot encoding (This will be used in model 1).

In [9]:
def int2onehot(dataset):
  y = []
  for example in dataset:
    if example:
      y.append(np.array([0,1]))
    else:
      y.append(np.array([1,0]))
  return np.array(y)
y_train_onehot = int2onehot(y_train)
y_dev_onehot = int2onehot(y_dev)
y_test_onehot = int2onehot(y_test)

print(y_train_onehot[0])
print(y_train_onehot[1])
print(y_train_onehot[2])
print(y_train_onehot[3])
print(y_train_onehot[4])

[1 0]
[1 0]
[1 0]
[1 0]
[0 1]


Now we have almost done the data preprocessing. Unlike the previous lab1-lab4, there are two distinct parts to the input x (tweet and topic - see lab 5) which we must input to the model. The easiest way is to input the tweet and topic as paired sentences into the model, concatenating them and separating them by the special BERT sentence-separator token [SEP] (available as tokenizer.sep_token). Then we can apply BERT directly.

In [10]:
# Please write your code to combine the tweet and topic into the following varibles
x_train_data = []
for record in train_original:
  x_train_data.append(record[3] + ' ' + tokenizer.sep_token + ' '+ record[1])

x_dev_data = []
for record in dev_original:
  x_dev_data.append(record[3] + ' ' + tokenizer.sep_token + ' '+ record[1])

x_test_data = []
for record in test_original:
  x_test_data.append(record[3] + ' ' + tokenizer.sep_token + ' '+ record[1])

x_train_int = []
x_train_masks = []
for record in x_train_data:
  ids,masks,segments = tokenize(record, tokenizer, pad_to_max_length=False)
  x_train_int.append(ids)
  x_train_masks.append(masks)

x_dev_int = []
x_dev_masks = []
for record in x_dev_data:
  ids,masks,segments = tokenize(record, tokenizer, pad_to_max_length=False)
  x_dev_int.append(ids)
  x_dev_masks.append(masks)

x_test_int = []
x_test_masks = []
for record in x_test_data:
  ids,masks,segments = tokenize(record, tokenizer, pad_to_max_length=False)
  x_test_int.append(ids)
  x_test_masks.append(masks)



# Tips: 
# 1) We can use the special token <SEP> to concatenate the tweets and topics.
# 2) After combine them, make sure they are paded.
x_train_int = np.array(keras.preprocessing.sequence.pad_sequences(x_train_int, padding='post', maxlen=128, value=0))
x_dev_int = np.array(keras.preprocessing.sequence.pad_sequences(x_dev_int, padding='post', maxlen=128, value=0))
x_test_int = np.array(keras.preprocessing.sequence.pad_sequences(x_test_int, padding='post', maxlen=128, value=0))
x_train_masks = np.array(keras.preprocessing.sequence.pad_sequences(x_train_masks, padding='post', maxlen=128, value=0))
x_dev_masks = np.array(keras.preprocessing.sequence.pad_sequences(x_dev_masks, padding='post', maxlen=128, value=0))
x_test_masks = np.array(keras.preprocessing.sequence.pad_sequences(x_test_masks, padding='post', maxlen=128, value=0))


# your code goes here



# Don't forget the to use np.array function to wrap the ouput of pad_sequences function
x_train_int_np = np.array(x_train_int)
x_train_masks_np = np.array(x_train_masks)
x_dev_int_np = np.array(x_dev_int)
x_dev_masks_np = np.array(x_dev_masks)
x_test_int_np = np.array(x_test_int)
x_test_masks_np = np.array(x_test_masks)


print(x_dev_int[0])
print(x_dev_masks[0],'\n')
print(x_dev_int_np[0])
print(x_dev_masks_np[0])


print('Few examples of text and topic included but separated by SEP token')
print(x_train_data[0])
print(x_dev_data[0])
print(x_test_data[0])

[  101  6108  1062  9794 16021 23091  2007 16839  9080 12863  7050  2000
  2745  4027  1024  6108  1062  4593  2587 16021 23091  2006  5095  1998
  1012  1012  8299  1024  1013  1013  1056  1012  2522  1013  1053  3501
  2683  2072  2549  8586  2615 18037   102  2745  4027   102     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

<font color = "#75a2eb"><b>In the above code we have concatenated the tweet and the topic with the bert seperator token in between them. We then passed the resulting string into the BERT tokenizer function the same way we did above. Finally we pad the sequences using the keras pad sequences function and wrap the result in a numpy array.</b></font>

In your report, show a few examples of your input, with text and topic included but separated by [SEP].

## Model 1: Prebuilt Sequence Classification



The huggingface transformer package provides many prebuilt models. First, let us try a basic text classification model based on DistilBERT. This first method is the standard way BERT models are used for text classification: we use the embedding of the special [CLS] token at the beginning of the sequence, and put it through a simple classifier layer to make the decision. The TFDistilBertForSequenceClassification class takes care of that for us.

The models with BERT are much bigger than our previous models. To run it faster, we can use TPU here. The detailed guideline about using TPU can be found from https://www.tensorflow.org/guide/tpu.

In [ ]:
from transformers import TFDistilBertForSequenceClassification, DistilBertConfig
import tensorflow as tf

distil_bert = 'distilbert-base-uncased'

config = DistilBertConfig(num_labels=2)
config.output_hidden_states = False

def create_TFDistilBertForSequenceClassification():
  transformer_model = TFDistilBertForSequenceClassification.from_pretrained(distil_bert, config = config)
  input_ids = tf.keras.layers.Input(shape=(128,), name='input_token', dtype='int32')
  input_masks_ids = tf.keras.layers.Input(shape=(128,), name='masked_token', dtype='int32')
  X = transformer_model(input_ids, input_masks_ids)
  return tf.keras.Model(inputs=[input_ids, input_masks_ids], outputs = X)

use_tpu = True
if use_tpu:
  # Create distribution strategy
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.experimental.TPUStrategy(tpu)

  # Create model on TPU:
  with strategy.scope():
    model = create_TFDistilBertForSequenceClassification()
    optimizer = keras.optimizers.Adam(lr=5e-5)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
else:
  model = create_TFDistilBertForSequenceClassification()
  model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Initializing the TPU system: grpc://10.74.187.130:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.74.187.130:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_token (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
masked_token (InputLayer)       [(None, 128)]        0                                            
__________________________________________________________________________________________________
tf_distil_bert_for_sequence_cla TFSequenceClassifier 66955010    input_token[0][0]                
                                                                 masked_token[0][0]               
Total params: 66,955,010
Trainable params: 66,955,010
Non-trainable params: 0
__________________________________________________________________________________________________


In [ ]:
history = model.fit([x_train_int_np,x_train_masks_np],
                    y_train_onehot,
                    epochs=30,
                    batch_size=512,
                    validation_data=([x_dev_int_np,x_dev_masks_np], y_dev_onehot),
                    verbose=1)

Epoch 1/30


35/35 [==============================] - ETA: 0s - loss: 1.2609 - accuracy: 0.6940WARNING:tensorflow:The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


35/35 [==============================] - 89s 1s/step - loss: 1.2444 - accuracy: 0.6964 - val_loss: 0.3806 - val_accuracy: 0.8317
Epoch 2/30
35/35 [==============================] - 7s 203ms/step - loss: 0.4413 - accuracy: 0.8174 - val_loss: 0.3829 - val_accuracy: 0.8445
Epoch 3/30
35/35 [==============================] - 7s 202ms/step - loss: 0.3102 - accuracy: 0.8767 - val_loss: 0.3688 - val_accuracy: 0.8415
Epoch 4/30
35/35 [==============================] - 7s 203ms/step - loss: 0.3195 - accuracy: 0.8787 - val_loss: 0.3654 - val_accuracy: 0.8475
Epoch 5/30
35/35 [==============================] - 7s 203ms/step - loss: 0.2777 - accuracy: 0.9071 - val_loss: 0.3629 - val_accuracy: 0.8528
Epoch 6/30
35/35 [==============================] - 7s 201ms/step - loss: 0.2161 - accuracy: 0.9237 - val_loss: 0.4019 - val_accuracy: 0.8543
Epoch 7/30
35/35 [==============================] - 7s 203ms/step - loss: 0.1836 - accuracy: 0.9363 - val_loss: 0.4569 - val_accuracy: 0.8528
Epoch 8/30
35/35 [=

In [ ]:
results = model.evaluate([x_test_int_np,x_test_masks_np], y_test_onehot)
print(results)

194/194 [==============================] - 8s 31ms/step - loss: 1.6490 - accuracy: 0.8524
[1.648979663848877, 0.8523848056793213]


In your report, show the accuracy you achieve, and compare it to what you achieved in Lab 5. Is it better or worse, and why?

<font color = "#75a2eb"><b>This model has 85% accuracy which is still greater than any model done in Lab 5. In Lab 5 we use Glove word embeddings while here we are using BERT. The main difference between them is that the words in word embeddings are independent of context. However a BERT is backed by a transformer and its main feature which is attention. This helps the model understand the contextual relationship between words. This helps this model get more accuracy compared to the models in Lab 5.</b></font>


## Model 2: Neural bag of words using BERT

In this model, we take the NBOW classifier from lab 4 (model3-1) and integrate BERT. Instead of averaging over word2vec or GloVe word vectors, we are averaging over the embedding representations produced by BERT - but otherwise, the classifier is the same.


In [ ]:
class GlobalAveragePooling1DMasked(GlobalAveragePooling1D):
    def call(self, x, mask=None):
        if mask != None:
            return K.sum(x, axis=1) / K.sum(mask, axis=1)
        else:
            return super().call(x)

In [ ]:
from transformers import TFDistilBertModel, DistilBertConfig

def get_BERT_layer():
  distil_bert = 'distilbert-base-uncased'
  config = DistilBertConfig(dropout=0.2, attention_dropout=0.2)
  config.output_hidden_states = False
  return TFDistilBertModel.from_pretrained(distil_bert, config = config)


In [ ]:

hdepth=16
MAX_SEQUENCE_LENGTH = 128
EMBED_SIZE=100


def create_bag_of_words_BERT():
  input_ids_in = tf.keras.layers.Input(shape=(128,), name='input_token', dtype='int32')
  input_masks_in = tf.keras.layers.Input(shape=(128,), name='masked_token', dtype='int32') 

  bert_embeddings = get_BERT_layer()
  embedded_sent = bert_embeddings(input_ids_in, attention_mask=input_masks_in)[0]

  pooled_sent=GlobalAveragePooling1DMasked()(embedded_sent)
  hidden_output=Dense(hdepth,input_shape=(MAX_SEQUENCE_LENGTH,EMBED_SIZE),activation='sigmoid',kernel_initializer='glorot_uniform')(pooled_sent) # Sigmoid
  label=Dense(1,input_shape=(hdepth,),activation='sigmoid',kernel_initializer='glorot_uniform')(hidden_output)
  return Model(inputs=[input_ids_in,input_masks_in], outputs=[label],name='Model2_BERT')

use_tpu = True
if use_tpu:
  # Create distribution strategy
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.experimental.TPUStrategy(tpu)

  # Create model
  with strategy.scope():
    model2 = create_bag_of_words_BERT()
    optimizer2 = keras.optimizers.Adam(lr=5e-5)
    model2.compile(optimizer=optimizer2, loss='binary_crossentropy', metrics=['accuracy'])
else:
  model2 = create_bag_of_words_BERT()
  model2.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model2.summary() 




INFO:tensorflow:Initializing the TPU system: grpc://10.74.187.130:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.74.187.130:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)
Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_projector', 'activation_13', 'vocab_transform', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertMo

Model: "Model2_BERT"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_token (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
masked_token (InputLayer)       [(None, 128)]        0                                            
__________________________________________________________________________________________________
tf_distil_bert_model (TFDistilB TFBaseModelOutput(la 66362880    input_token[0][0]                
                                                                 masked_token[0][0]               
__________________________________________________________________________________________________
global_average_pooling1d_masked (None, 768)          0           tf_distil_bert_model[0]

In [ ]:

history = model2.fit([x_train_int_np,x_train_masks_np],
                    y_train,
                    epochs=30,
                    batch_size=512,
                    validation_data=([x_dev_int_np,x_dev_masks_np], y_dev),
                    verbose=1)

Epoch 1/30


35/35 [==============================] - ETA: 0s - loss: 0.5522 - accuracy: 0.6857WARNING:tensorflow:The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


35/35 [==============================] - 91s 1s/step - loss: 0.5495 - accuracy: 0.6887 - val_loss: 0.3799 - val_accuracy: 0.8483
Epoch 2/30
35/35 [==============================] - 7s 204ms/step - loss: 0.3238 - accuracy: 0.8893 - val_loss: 0.4027 - val_accuracy: 0.8347
Epoch 3/30
35/35 [==============================] - 7s 203ms/step - loss: 0.2825 - accuracy: 0.9161 - val_loss: 0.3561 - val_accuracy: 0.8679
Epoch 4/30
35/35 [==============================] - 7s 202ms/step - loss: 0.2428 - accuracy: 0.9408 - val_loss: 0.3752 - val_accuracy: 0.8619
Epoch 5/30
35/35 [==============================] - 7s 202ms/step - loss: 0.2101 - accuracy: 0.9588 - val_loss: 0.3963 - val_accuracy: 0.8536
Epoch 6/30
35/35 [==============================] - 7s 201ms/step - loss: 0.1940 - accuracy: 0.9675 - val_loss: 0.3977 - val_accuracy: 0.8558
Epoch 7/30
35/35 [==============================] - 7s 201ms/step - loss: 0.1864 - accuracy: 0.9695 - val_loss: 0.4070 - val_accuracy: 0.8551
Epoch 8/30
35/35 [=

In [ ]:
results = model2.evaluate([x_test_int_np,x_test_masks_np], y_test)
print(results)

194/194 [==============================] - 8s 30ms/step - loss: 0.3575 - accuracy: 0.8718
[0.3575366139411926, 0.8717865347862244]


In your report, show the accuracy you achieve, and compare it to what you achieved with Model 1 and in Lab 5. Is it better or worse, and why?

<font color = "#75a2eb"><b>This model also has the advantage of BERT which understood the contextual relationship between words as opposed to word embeddings in Lab 5. This model also has the help of Neural bag of words which help the model gain an accuracy of 87%. NBOW is able to learn the word importance weights which is a new weighted sum composition of the BERT word vectors. This composition of vectors helps the model in easy classification.</b></font>

## Model 3: CNN or LSTM with BERT

Please following model2 to finish a CNN or LSTM model with BERT. 

In [ ]:
# your code goes here
#with CNN

hdepth=16
MAX_SEQUENCE_LENGTH = 128
EMBED_SIZE=100


def create_bag_of_words_BERT():
  input_ids_in = tf.keras.layers.Input(shape=(128,), name='input_token', dtype='int32')
  input_masks_in = tf.keras.layers.Input(shape=(128,), name='masked_token', dtype='int32') 

  bert_embeddings = get_BERT_layer()
  embedded_sent = bert_embeddings(input_ids_in, attention_mask=input_masks_in)[0]
  cnn_layer = keras.layers.Conv1D(100, 6)(embedded_sent)

  pooled_sent=GlobalAveragePooling1DMasked()(cnn_layer)
  hidden_output=Dense(hdepth,input_shape=(MAX_SEQUENCE_LENGTH,EMBED_SIZE),activation='sigmoid',kernel_initializer='glorot_uniform')(pooled_sent) # Sigmoid
  label=Dense(1,input_shape=(hdepth,),activation='sigmoid',kernel_initializer='glorot_uniform')(hidden_output)
  return Model(inputs=[input_ids_in,input_masks_in], outputs=[label],name='Model2_BERT')

use_tpu = True
if use_tpu:
  # Create distribution strategy
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.experimental.TPUStrategy(tpu)

  # Create model
  with strategy.scope():
    model2 = create_bag_of_words_BERT()
    optimizer2 = keras.optimizers.Adam(lr=5e-5)
    model2.compile(optimizer=optimizer2, loss='binary_crossentropy', metrics=['accuracy'])
else:
  model2 = create_bag_of_words_BERT()
  model2.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model2.summary() 




INFO:tensorflow:Initializing the TPU system: grpc://10.74.187.130:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.74.187.130:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)
Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_projector', 'activation_13', 'vocab_transform', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertMo

Model: "Model2_BERT"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_token (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
masked_token (InputLayer)       [(None, 128)]        0                                            
__________________________________________________________________________________________________
tf_distil_bert_model_2 (TFDisti TFBaseModelOutput(la 66362880    input_token[0][0]                
                                                                 masked_token[0][0]               
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 123, 100)     460900      tf_distil_bert_model_2[

<font color = "#75a2eb"><b>We have added a Convolutional 2D layer along with the previously created BERT layer to check and see if there's any improvement in the performance.</b></font>

In [ ]:

history = model2.fit([x_train_int_np,x_train_masks_np],
                    y_train,
                    epochs=30,
                    batch_size=512,
                    validation_data=([x_dev_int_np,x_dev_masks_np], y_dev),
                    verbose=1)

Epoch 1/30


35/35 [==============================] - ETA: 0s - loss: 0.5208 - accuracy: 0.7288WARNING:tensorflow:The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


35/35 [==============================] - 93s 1s/step - loss: 0.5184 - accuracy: 0.7309 - val_loss: 0.3620 - val_accuracy: 0.8377
Epoch 2/30
35/35 [==============================] - 8s 219ms/step - loss: 0.2846 - accuracy: 0.8938 - val_loss: 0.3397 - val_accuracy: 0.8589
Epoch 3/30
35/35 [==============================] - 7s 204ms/step - loss: 0.2239 - accuracy: 0.9290 - val_loss: 0.3471 - val_accuracy: 0.8536
Epoch 4/30
35/35 [==============================] - 7s 204ms/step - loss: 0.1802 - accuracy: 0.9555 - val_loss: 0.3588 - val_accuracy: 0.8498
Epoch 5/30
35/35 [==============================] - 7s 204ms/step - loss: 0.1700 - accuracy: 0.9581 - val_loss: 0.3908 - val_accuracy: 0.8362
Epoch 6/30
35/35 [==============================] - 7s 203ms/step - loss: 0.1536 - accuracy: 0.9673 - val_loss: 0.3906 - val_accuracy: 0.8475
Epoch 7/30
35/35 [==============================] - 7s 203ms/step - loss: 0.1374 - accuracy: 0.9749 - val_loss: 0.3935 - val_accuracy: 0.8521
Epoch 8/30
35/35 [=

In [ ]:
results = model2.evaluate([x_test_int_np,x_test_masks_np], y_test)
print(results)

194/194 [==============================] - 8s 31ms/step - loss: 0.4243 - accuracy: 0.8584
[0.42431047558784485, 0.8583670258522034]


In your report, show the accuracy you achieve, and compare it to what you achieved with Model 1 and Model 2. Is it better or worse, and why?

<font color = "#75a2eb"><b> The CNN was on par with Model 1 but slightly worse than Model 2. However, the loss for CNN model is much less compared to model 1. This is because the CNN concatenates words within a single window as opposed to all the words done by NBOW and assigns equal importance to each word. Although the result keeps fluctuating and the CNN was able to perform better than Model 2 in some cases.</b></font>